In [9]:
test = [(423, 95), (158, 207), (72, 92), (124, 384), (382, 301), (568, 353), (517, 209), (236, 252)]

In [41]:
import numpy as np

path = []
def find_closest_hotspot(curr_hotspot, rest_hotspots):    
    if len(rest_hotspots) < 1:
        return path
    
    rest_hotspots = np.array(rest_hotspots)
    distances = np.sqrt(np.sum((rest_hotspots - curr_hotspot) ** 2, axis=1))
    closest_hotspot = rest_hotspots[np.argmin(distances)]
    path.append(closest_hotspot)
    
    _ = find_closest_hotspot(closest_hotspot, np.delete(rest_hotspots, np.argmin(distances), axis=0))

In [43]:
find_closest_hotspot(test[0], np.array(test[1:]))
path

[array([517, 209]),
 array([568, 353]),
 array([382, 301]),
 array([236, 252]),
 array([158, 207]),
 array([72, 92]),
 array([124, 384]),
 array([517, 209]),
 array([568, 353]),
 array([382, 301]),
 array([236, 252]),
 array([158, 207]),
 array([72, 92]),
 array([124, 384])]

In [44]:
from itertools import combinations

def held_karp(dists):
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset_size in range(2, n):
        for subset in combinations(range(1, n), subset_size):
            # Set bits for all nodes in this subset
            bits = 0
            for bit in subset:
                bits |= 1 << bit

            # Find the lowest cost to get to this subset
            for k in subset:
                prev = bits & ~(1 << k)

                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + dists[m][k], m))
                C[(bits, k)] = min(res)

    # We're interested in all bits but the least significant (the start state)
    bits = (2**n - 1) - 1

    # Calculate optimal cost
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res)

    # Backtrack to find full path
    path = []
    for i in range(n - 1):
        path.append(parent)
        new_bits = bits & ~(1 << parent)
        _, parent = C[(bits, parent)]
        bits = new_bits

    # Add implicit return to start
    path.append(0)

    return opt, list(reversed(path))

In [45]:
held_karp(test)

IndexError: tuple index out of range